In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#Unzip the Dataset
!unzip "/content/drive/MyDrive/ATCC_DATASET1.zip" -d "/content/atcc_videos"


Archive:  /content/drive/MyDrive/ATCC_DATASET1.zip
  inflating: /content/atcc_videos/Talaimari/North-East.mp4  
  inflating: /content/atcc_videos/Talaimari/North-Weast.mp4  
  inflating: /content/atcc_videos/Talaimari/Weast.mp4  
  inflating: /content/atcc_videos/Talaimari/east.mp4  
  inflating: /content/atcc_videos/Vodra/North.mp4  
  inflating: /content/atcc_videos/Vodra/South.mp4  
  inflating: /content/atcc_videos/Vodra/weast.mp4  


In [ ]:
import os
os.listdir("/content/atcc_videos")


['Talaimari', 'Vodra']

In [ ]:
#Set Dataset & Output Paths

import os

TALIMARI_DIR = "/content/atcc_videos/Talaimari"
VODRA_DIR    = "/content/atcc_videos/Vodra"

BASE_OUTPUT = "/content/drive/MyDrive/ATCC"
FRAME_DIR   = f"{BASE_OUTPUT}/frames"
OUTPUT_DIR  = f"{BASE_OUTPUT}/output"
CSV_PATH    = f"{BASE_OUTPUT}/atcc_results.csv"

os.makedirs(FRAME_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Talimari:", os.listdir(TALIMARI_DIR))
print("Vodra:", os.listdir(VODRA_DIR))



Talimari: ['North-East.mp4', 'Weast.mp4', 'east.mp4', 'North-Weast.mp4']
Vodra: ['North.mp4', 'weast.mp4', 'South.mp4']


In [ ]:
# Zip the output images before downloading
!zip -r atcc_output_images.zip {OUTPUT_DIR}

from google.colab import files
files.download("atcc_output_images.zip")

In [ ]:
#Install Required Libraries

!pip install -q ultralytics opencv-python pandas tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.9 MB/s eta 0:00:00


In [ ]:
#Load YOLO Model
from ultralytics import YOLO
model = YOLO("yolov8n.pt")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
#Extract Frames from ALL Videos (ZIP DATASET)

import cv2
from tqdm import tqdm

def extract_frames(video_path, skip=5):
    cap = cv2.VideoCapture(video_path)
    name = os.path.basename(video_path).rsplit(".", 1)[0]
    i, saved = 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if i % skip == 0:
            out_path = f"{FRAME_DIR}/{name}_{saved}.jpg"
            cv2.imwrite(out_path, frame)
            saved += 1
        i += 1

    cap.release()
    print(f"Extracted {saved} frames from {name}")

# Collect all videos from both folders
all_videos = []

for folder in [TALIMARI_DIR, VODRA_DIR]:
    videos = [os.path.join(folder, f) for f in os.listdir(folder)
              if f.lower().endswith(('.mp4','.avi','.mov'))]
    all_videos.extend(videos)

print("Total videos found:", len(all_videos))

for v in tqdm(all_videos):
    extract_frames(v, skip=5)

print("✅ Frame extraction completed")


Total videos found: 7


 14%|█▍        | 1/7 [02:10<13:04, 130.83s/it]

Extracted 1022 frames from North-East


 29%|██▊       | 2/7 [08:20<22:36, 271.28s/it]

Extracted 1021 frames from Weast


...

Extracted 722 frames from North


 86%|████████▌ | 6/7 [17:16<02:00, 120.24s/it]

Extracted 722 frames from weast


100%|██████████| 7/7 [18:05<00:00, 155.11s/it]

Extracted 721 frames from South
✅ Frame extraction completed


In [ ]:
#ATCC Detection + Vehicle Count + CSV

import pandas as pd
from collections import Counter
import cv2
from tqdm import tqdm

results_table = []

frames = [f for f in os.listdir(FRAME_DIR) if f.endswith(".jpg")]

for f in tqdm(frames):
    img_path = f"{FRAME_DIR}/{f}"
    img = cv2.imread(img_path)

    results = model(img)
    r = results[0]

    counts = Counter()
    for box in r.boxes:
        cls = int(box.cls)
        name = r.names[cls]
        counts[name] += 1

    results_table.append({"frame": f, **counts})

    annotated = r.plot()
    cv2.imwrite(f"{OUTPUT_DIR}/{f}", annotated)

df = pd.DataFrame(results_table).fillna(0)
df.to_csv(CSV_PATH, index=False)

df.head()


  0%|          | 0/6250 [00:00<?, ?it/s]


0: 384x640 3 persons, 4 trucks, 79.8ms
Speed: 15.2ms preprocess, 79.8ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)


  0%|          | 1/6250 [00:02<4:31:54,  2.61s/it]


0: 384x640 6 persons, 1 bicycle, 2 cars, 1 motorcycle, 3 buss, 2 trucks, 7.6ms
Speed: 2.3ms preprocess, 7.6ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 2 cars, 1 bus, 2 trucks, 6.7ms
Speed: 2.4ms preprocess, 6.7ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)


  0%|          | 3/6250 [00:02<1:14:43,  1.39it/s]

...


0: 384x640 (no detections), 16.5ms
Speed: 2.2ms preprocess, 16.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.6ms
Speed: 2.5ms preprocess, 25.6ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)


100%|█████████▉| 6248/6250 [08:46<00:00, 11.44it/s]


0: 384x640 (no detections), 14.1ms
Speed: 3.6ms preprocess, 14.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.1ms
Speed: 4.3ms preprocess, 12.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


100%|██████████| 6250/6250 [08:46<00:00, 11.86it/s]


,frame,person,truck,bus,car,motorcycle,bicycle,bench,train,handbag,...,elephant,tie,tennis racket,banana,dog,clock,horse,cell phone,cup,snowboard
0,east_640.jpg,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,east_674.jpg,6.0,2.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,east_678.jpg,4.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,east_632.jpg,4.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,east_691.jpg,1.0,3.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from google.colab import files
files.download(CSV_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r atcc_output_images.zip /content/drive/MyDrive/ATCC

Streaming output truncated to the last 5000 lines.
  adding: content/drive/MyDrive/ATCC/output/Weast_291.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_292.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_293.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_294.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_295.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_296.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_297.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_298.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_299.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_300.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_301.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_302.jpg (deflated 1%)
  adding: content/drive/MyDrive/ATCC/output/Weast_303.jpg (deflated 1%)
  adding: con